In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import codecs
from tabulate import tabulate
import re
import string
from wordcloud import WordCloud
import nltk
from nltk.tokenize import word_tokenize
from num2words import num2words

nltk.download('words')
nltk.download('punkt')

from sklearn.model_selection import train_test_split
import numpy as np
import random
from datasets import Dataset, DatasetDict

import spacy

import torchtext
import torch
import torch.optim as optim
from tqdm import tqdm
import torch.nn as nn

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
e:\TU\SoSe24\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# Task 1: Data exploration and Sampling

In [ ]:
# Function to read text files safely
def read_text_file(file_path):
    with codecs.open(file_path, 'r', encoding='utf-8') as file:
        sentences = file.readlines()
    return sentences

# Function to calculate most frequent word
def most_frequent_word(text):
    words = text.split()
    word_freq = Counter(words)
    return word_freq.most_common(1)[0][0]

# Function to calculate unique words count
def count_unique_words(text):
    words = text.split()
    return len(set(words))

# Function to calculate numeral frequencies
def count_numerals(text):
    numeral_count = Counter(char for char in text if char.isdigit())
    return numeral_count

# File paths
en_file_path = 'de-en/europarl-v7.de-en.en'  # English language file
de_file_path = 'de-en/europarl-v7.de-en.de'  # German language file

# Read the files
en_sentences = read_text_file(en_file_path)
de_sentences = read_text_file(de_file_path)

# Create a DataFrame
df = pd.DataFrame({'en': en_sentences, 'de': de_sentences})

## Stats

In [ ]:
# Basic statistics
num_sentences = len(df)
en_lengths = df['en'].str.len()
de_lengths = df['de'].str.len()
length_diff = en_lengths - de_lengths

# Calculate total number of words, unique words, and average word length
def calculate_word_stats(text):
    words = text.split()
    num_words = len(words)
    unique_words = count_unique_words(text)
    avg_word_length = sum(len(word) for word in words) / num_words
    return num_words, unique_words, avg_word_length

en_num_words, unique_en_words, awl_en = calculate_word_stats(' '.join(df['en']))
de_num_words, unique_de_words, awl_de = calculate_word_stats(' '.join(df['de']))

# Calculate most frequent words
most_freq_word_en = most_frequent_word(' '.join(df['en']))
most_freq_word_de = most_frequent_word(' '.join(df['de']))

# Calculate numeral frequencies
numeral_freq_en = count_numerals(' '.join(df['en']))
numeral_freq_de = count_numerals(' '.join(df['de']))

# Summary statistics
summary_stats = [
    ['Number of sentences', num_sentences],
    ['Total words (English)', en_num_words],
    ['Total words (German)', de_num_words],
    ['Unique words (English)', unique_en_words],
    ['Unique words (German)', unique_de_words],
    ['Average word length (English)', awl_en],
    ['Average word length (German)', awl_de],
    ['Average sentence length (English)', en_lengths.mean()],
    ['Average sentence length (German)', de_lengths.mean()],
    ['Average sentence length difference (English - German)', length_diff.mean()],
    ['Most frequent word (English)', most_freq_word_en],
    ['Most frequent word (German)', most_freq_word_de]
    #['Numerals frequency (English)', numeral_freq_en],
    #['Numerals frequency (German)', numeral_freq_de]
]

# Print summary statistics
print(tabulate(summary_stats, headers=['Statistic', 'Value'], tablefmt='pretty'))

## Display word clouds

In [ ]:
# Generate and display word clouds
def generate_word_cloud(text, title):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(title)
    plt.axis('off')
    plt.show()

# Create word clouds for English and German text
generate_word_cloud(' '.join(df['en']), 'Most Common Words in English')
generate_word_cloud(' '.join(df['de']), 'Most Common Words in German')

## Distribution of Average Word Length

In [ ]:
df['en_sentence_length'] = df['en'].str.len()
df['de_sentence_length'] = df['de'].str.len()

df['en_num_words'] = df['en'].str.split().apply(len)
df['de_num_words'] = df['de'].str.split().apply(len)

df['en_avg_word_length'] = df['en_sentence_length'] / df['en_num_words']
df['de_avg_word_length'] = df['de_sentence_length'] / df['de_num_words']

# Plotting with Seaborn
plt.figure(figsize=(4, 4))

# Distribution of Average Word Length
plt.subplot(1, 1, 1)
sns.histplot(df['en_avg_word_length'], kde=True, color='blue', label='English')
sns.histplot(df['de_avg_word_length'], kde=True, color='green', label='German')
plt.title('Distribution of Average Word Length')
plt.xlabel('Average Word Length (Characters)')
plt.legend()

# Show plot
plt.tight_layout()
plt.show()


# Task 2: Pre-processing

In [ ]:
# Randomly select 10% of the data
df_sampled = df.sample(frac=0.10, random_state=42)

# Preprocessing steps
def preprocess_text(df, column, lang, replacements):
    df[column] = df[column].str.lower()
    df[column] = df[column].apply(remove_number_lists)
    df[column] = df[column].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
    df[column] = df[column].apply(remove_specific_characters, args=(replacements,))
    df[column] = df[column].apply(remove_whitespace_before_numbers)
    df[column] = df[column].apply(remove_report_number)
    df[column] = df[column].apply(lambda x: ' '.join(x.split()))
    df[column] = df[column].apply(convert_numbers_to_words, args=(lang,))
    return df

def remove_number_lists(text):
    pattern = r'\b\d+(?:,\s*\d+)*\b'
    return re.sub(pattern, '', text).strip()

def remove_specific_characters(text, replacements):
    for key, value in replacements.items():
        text = text.replace(key, value)
    text = re.sub(r'[-„“‟”–…‘’´­•—‚‘υπέρœ―]', '', text)
    return ' '.join(text.split())

def remove_whitespace_before_numbers(text):
    return re.sub(r'(\d)\s+(\d)', r'\1\2', text)

def remove_report_number(text):
    text = re.sub(r'\b[a-z]\d+\b', '', text)
    return ' '.join(text.split())

def convert_numbers_to_words(sentence, lang='en'):
    words = word_tokenize(sentence)
    converted_words = []
    for word in words:
        try:
            number = int(word)
            if 0 <= number <= 999999999:
                converted_words.append(num2words(number, lang=lang))
            else:
                converted_words.append(word)
        except (ValueError, NotImplementedError):
            converted_words.append(word)
    return ' '.join(converted_words)

def write_files(df):
    columns_to_save = {'en': 'en.txt', 'de': 'de.txt'}
    for column, filename in columns_to_save.items():
        df[column].to_csv(filename, index=False, header=False, encoding='utf-8')
    print("Columns have been saved to separate files.")

# Define replacements for 'en' and 'de'
replacements = {
    'en': {'½': 'one half', '¾': 'three quarters', '£': 'pound', '°': 'degree', '§': 'section'},
    'de': {'½': 'ein halb', '¾': 'drei viertel', '€': 'euro', '°': 'grad', '§': 'abschnitt'}
}

# Apply preprocessing to 'en' and 'de' columns
df_sampled = preprocess_text(df_sampled, 'en', 'en', replacements['en'])
df_sampled = preprocess_text(df_sampled, 'de', 'de', replacements['de'])

# Strip empty lines and their correspondences
df_sampled = df_sampled[(df_sampled['en'].str.strip() != '') & (df_sampled['de'].str.strip() != '')]

# Write the processed data to files
write_files(df_sampled)

# Task 3: Neural Machine Translation

## Load 10 % of data

In [3]:
# Read and preprocess data
with open("de.txt", 'r', encoding='utf-8') as file1: 
    de_texts = [line.strip() for line in file1]
with open("en.txt", 'r', encoding='utf-8') as file2: 
    en_texts = [line.strip() for line in file2]

# Create a DataFrame from the lists
df_sampled = pd.DataFrame({'de': de_texts, 'en': en_texts})
df_sampled = df_sampled.sample(frac=0.05, random_state=1).reset_index(drop=True)
df_sampled.tail()
print(df_sampled.head())

                                                  de  \
0  rückwürfe von bis zu zwei millionen tonnen ges...   
1  ferner zeigt sie nützliche marktinstrumente zu...   
2  dies bringt jedoch auch höhere kosten für die ...   
3            bedeutende reaktionsfähigkeit potenzial   
4  ein tätigwerden auf dem gebiet der vermarktung...   

                                                  en  
0  discarding nearly two million tonnes of health...  
1  they can also point out useful market instrume...  
2   however it will mean more costs for the consumer  
3                   significant reactivity potential  
4  the need for action on the marketing of constr...  


## Split data. 20% test and rest into train and val split

In [4]:
test_size = 0.2  # 20% for test set
train_val_df, test_df = train_test_split(df_sampled, test_size=test_size, random_state=42)

# Further split train/validation:
val_size = 0.08  # 8% of remaining data for validation
train_df, val_df = train_test_split(train_val_df, test_size=val_size, random_state=42)

print("Train data shape:", train_df.shape)
print("Validation data shape:", val_df.shape)
print("Test data shape:", test_df.shape)

train_df.head()

Train data shape: (7006, 2)
Validation data shape: (610, 2)
Test data shape: (1904, 2)


,de,en
4997,wir werden ab mai eine gemeinschaft mit millio...,from the first of may we shall be a community ...
5944,bericht von frau maría rodríguez ramos im name...,report by maría rodríguez ramos on behalf of t...
9334,für das europäische projekt von morgen sind wi...,for the european project of the future we are ...
2102,schriftlich sv der bericht über die gemeinsame...,in writing sv the report on the common agricul...
7075,frauen und mädchen werden unterdrückt insbeson...,women and girls suffer oppression particularly...


## Create DatasetDict from train_df, val_df and test_df

In [5]:
# Convert DataFrames to Dataset objects
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Create DatasetDict
dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

# Print the DatasetDict
print(dataset)

train_data, valid_data, test_data = (
    dataset["train"],
    dataset["validation"],
    dataset["test"],
)

DatasetDict({
    train: Dataset({
        features: ['de', 'en', '__index_level_0__'],
        num_rows: 7006
    })
    validation: Dataset({
        features: ['de', 'en', '__index_level_0__'],
        num_rows: 610
    })
    test: Dataset({
        features: ['de', 'en', '__index_level_0__'],
        num_rows: 1904
    })
})


In [6]:
#!python -m spacy download en_core_web_sm
#!python -m spacy download de_core_news_sm

## Load tokenizers

In [6]:
en_nlp = spacy.load("en_core_web_sm")
de_nlp = spacy.load("de_core_news_sm")

## Tokenize train, val and test data sets

In [7]:
sos_token = "<sos>"
eos_token = "<eos>"

def tokenize_example(example, max_length, sos_token, eos_token):
    en_chars = list(example["en"])[:max_length]
    de_chars = list(example["de"])[:max_length]

    en_chars = [sos_token] + en_chars + [eos_token]
    de_chars = [sos_token] + de_chars + [eos_token]

    return {"en_tokens": en_chars, "de_tokens": de_chars}

max_length = 1000

fn_kwargs = {
    "max_length": max_length,
    "sos_token": sos_token,
    "eos_token": eos_token,
}

train_data = train_data.map(tokenize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(tokenize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(tokenize_example, fn_kwargs=fn_kwargs)

Map: 100%|██████████| 1904/1904 [00:00<00:00, 7936.58 examples/s]


## Create vocabularies with words and special tokens

In [8]:
min_freq = 3 # for a character to appear
unk_token = "<unk>"
pad_token = "<pad>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]

# Tokens ONLY from train set
all_en_tokens = train_data["en_tokens"]
all_de_tokens = train_data["de_tokens"]

# Build vocabularies
en_vocab = torchtext.vocab.build_vocab_from_iterator(
    all_en_tokens,
    min_freq=min_freq,
    specials=special_tokens,
)

de_vocab = torchtext.vocab.build_vocab_from_iterator(
    all_de_tokens,
    min_freq=min_freq,
    specials=special_tokens,
)

## Get indices of special tokens and set return index for non existing word in our vocabulary

In [9]:
unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

en_vocab.set_default_index(unk_index)
de_vocab.set_default_index(unk_index)

x = 8
de_vocab.get_itos()[:x], en_vocab.get_itos()[:x]

(['<unk>', '<pad>', '<sos>', '<eos>', 'e', ' ', 'n', 'i'],
 ['<unk>', '<pad>', '<sos>', '<eos>', ' ', 'e', 't', 'o'])

## word2index

In [10]:
def numericalize_example(example, en_vocab, de_vocab):
    # Convert token lists to their corresponding indices
    en_ids = en_vocab.lookup_indices(example["en_tokens"])
    de_ids = de_vocab.lookup_indices(example["de_tokens"])
    return {"en_ids": en_ids, "de_ids": de_ids}

# Dictionary of keyword arguments for the function
fn_kwargs = {"en_vocab": en_vocab, "de_vocab": de_vocab}

# Apply the numericalization function to each split of the dataset
train_data = train_data.map(numericalize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(numericalize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(numericalize_example, fn_kwargs=fn_kwargs)

Map: 100%|██████████| 1904/1904 [00:01<00:00, 1732.36 examples/s]


## Indices of words as torch tensor

In [11]:
# Convert ids to torch tensors
data_type = "torch"
format_columns = ["en_ids", "de_ids"]

train_data = train_data.with_format(
    type=data_type, columns=format_columns, output_all_columns=True
)

valid_data = valid_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

test_data = test_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

## Padding

In [12]:
# Padding
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_en_ids = [example["en_ids"] for example in batch]
        batch_de_ids = [example["de_ids"] for example in batch]
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
        batch_de_ids = nn.utils.rnn.pad_sequence(batch_de_ids, padding_value=pad_index)
        batch = {
            "en_ids": batch_en_ids,
            "de_ids": batch_de_ids,
        }
        return batch

    return collate_fn

## DataLoaders with collate_fn function

In [13]:
# Will give DataLoaders based on provided dataset
def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader

## Crate data loaders

In [14]:
batch_size = 64

train_data_loader = get_data_loader(train_data, batch_size, pad_index, shuffle=True)
valid_data_loader = get_data_loader(valid_data, batch_size, pad_index, shuffle=False)
test_data_loader = get_data_loader(test_data, batch_size, pad_index, shuffle=False)

## RNN based seq2seq architecture

In [15]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout, batch_first=False)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        outputs, (hidden, cell) = self.rnn(embedded)
        return hidden, cell
    
class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout, batch_first=False)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        prediction = self.fc_out(output.squeeze(0))
        return prediction, hidden, cell
    
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio):
        batch_size = trg.shape[1]
        trg_length = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(self.device)
        hidden, cell = self.encoder(src)
        input = trg[0, :]
        for t in range(1, trg_length):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg[t] if teacher_force else top1
        return outputs

## Set parameters

In [16]:
input_dim = len(en_vocab)
output_dim = len(de_vocab)

encoder_embedding_dim = 300
decoder_embedding_dim = 300
hidden_dim = 256
n_layers = 2
encoder_dropout = 0.5
decoder_dropout = 0.5

encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    hidden_dim,
    n_layers,
    encoder_dropout,
)

decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    hidden_dim,
    n_layers,
    decoder_dropout,
)

## Train and evaluation loops

In [17]:
def train_fn(
    model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device
):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(tqdm(data_loader)):
        src = batch["en_ids"].to(device)
        trg = batch["de_ids"].to(device)

        optimizer.zero_grad()
        
        output = model(src, trg, teacher_forcing_ratio)

        output_dim = output.shape[-1]

        output = output[1:].view(-1, output_dim)

        trg = trg[1:].view(-1)

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

def evaluate_fn(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(tqdm(data_loader)):
            src = batch["en_ids"].to(device)
            trg = batch["de_ids"].to(device)

            output = model(src, trg, 0)

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)

            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(data_loader)

## Load GloVe Embedding Matrices. Train with GloVe embeddings. Evaluate with GloVe embeddings.

In [18]:
os.makedirs("embeddings", exist_ok=True)
import torch
from torchtext.vocab import Vocab

# Create glove embedding matrices
def load_glove_embeddings(glove_file, vocab: Vocab, embedding_dim: int):
    # Initialize the embedding matrix with small random values using PyTorch
    embedding_matrix = torch.randn((len(vocab), embedding_dim)) * 0.01
    
    # Load pre-trained GloVe embeddings
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.split()
            word = parts[0]
            if word in vocab:  # Check if the word is in the vocabulary
                idx = vocab.get_stoi()[word]
                embedding_vector = torch.tensor([float(val) for val in parts[1:]], dtype=torch.float32)
                embedding_matrix[idx] = embedding_vector
    
    # Initialize special token embeddings
    # 0: '<unk>', 1: '<pad>', 2: '<sos>', 3: '<eos>'
    embedding_matrix[0] = torch.randn(embedding_dim) * 0.1  # Small random values for unknown
    embedding_matrix[1] = torch.zeros(embedding_dim)  # Zero initialization for padding token
    embedding_matrix[2] = torch.randn(embedding_dim) * 0.1  # Small random values (sos)
    embedding_matrix[3] = torch.randn(embedding_dim) * 0.1  # Small random values (eos)
    
    return embedding_matrix

embedding_dim = 300
en_glove_file = 'glove.6B.300d.txt'
de_glove_file = 'glove.de.txt' # https://www.deepset.ai/german-word-embeddings

char_en_gembedding_matrix = load_glove_embeddings(en_glove_file, en_vocab, embedding_dim)
char_de_gembedding_matrix = load_glove_embeddings(de_glove_file, de_vocab, embedding_dim)

# Save the embedding matrices
# torch.save(char_en_gembedding_matrix, 'embeddings/char_en_gembedding_matrix.pt')
# torch.save(char_de_gembedding_matrix, 'embeddings/char_de_gembedding_matrix.pt')

# Load the embedding matrices
# char_en_gembedding_matrix = torch.load('embeddings/char_en_gembedding_matrix.pt')
# char_de_gembedding_matrix = torch.load('embeddings/char_de_gembedding_matrix.pt')

### Train English to German model with GloVe embeddings

In [19]:
os.makedirs("models", exist_ok=True)

import gc
gc.collect()
torch.cuda.empty_cache()

model = Seq2Seq(encoder, decoder, device).to(device)

# Load our preptrained GloVe embeddings
encoder.embedding.weight.data.copy_(char_en_gembedding_matrix)
decoder.embedding.weight.data.copy_(char_de_gembedding_matrix)

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

n_epochs = 10
clip = 1.0
teacher_forcing_ratio = 0.5
best_valid_loss = float("inf")

for epoch in range(n_epochs):
    train_loss = train_fn(
        model,
        train_data_loader,
        optimizer,
        criterion,
        clip,
        teacher_forcing_ratio,
        device,
    )
    valid_loss = evaluate_fn(
        model,
        valid_data_loader,
        criterion,
        device,
    )
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "models/character_model_glove_en_de.pt")
    print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")
    print(f"Batch {epoch+1} done \n")

100%|██████████| 10/10 [00:02<00:00,  3.45it/s]


	Train Loss:   2.972 | Train PPL:  19.531
	Valid Loss:   2.902 | Valid PPL:  18.212
Batch 1 done 



100%|██████████| 10/10 [00:04<00:00,  2.45it/s]


	Train Loss:   2.792 | Train PPL:  16.306
	Valid Loss:   2.988 | Valid PPL:  19.844
Batch 2 done 



100%|██████████| 10/10 [00:04<00:00,  2.39it/s]


...

	Train Loss:   2.469 | Train PPL:  11.809
	Valid Loss:   2.972 | Valid PPL:  19.523
Batch 8 done 



100%|██████████| 10/10 [00:02<00:00,  3.57it/s]


	Train Loss:   2.446 | Train PPL:  11.545
	Valid Loss:   2.963 | Valid PPL:  19.352
Batch 9 done 



100%|██████████| 10/10 [00:02<00:00,  3.52it/s]

	Train Loss:   2.426 | Train PPL:  11.317
	Valid Loss:   2.948 | Valid PPL:  19.073
Batch 10 done 



### Evaluate English to German with BLEU and ROUGH scores with GloVe embeddings.

In [20]:
model.load_state_dict(torch.load("models/character_model_glove_en_de.pt"))
test_loss = evaluate_fn(model, test_data_loader, criterion, device)

print(f"| Test Loss: {test_loss:.3f} | Test PPL: {np.exp(test_loss):7.3f} |")

100%|██████████| 30/30 [00:13<00:00,  2.26it/s]

| Test Loss: 2.904 | Test PPL:  18.241 |


In [21]:
def translate_sentence(
    sentence,
    model,
    en_nlp,
    de_nlp,
    en_vocab,
    de_vocab,
    lower,
    sos_token,
    eos_token,
    device,
    max_output_length=25,
):
    model.eval()
    with torch.no_grad():
        if isinstance(sentence, str):
            tokens = [token.text for token in en_nlp.tokenizer(sentence)]
        else:
            tokens = [token for token in sentence]
        if lower:
            tokens = [token.lower() for token in tokens]
        tokens = [sos_token] + tokens + [eos_token]
        ids = en_vocab.lookup_indices(tokens)
        tensor = torch.LongTensor(ids).unsqueeze(-1).to(device)
        hidden, cell = model.encoder(tensor)
        inputs = de_vocab.lookup_indices([sos_token])
        for _ in range(max_output_length):
            inputs_tensor = torch.LongTensor([inputs[-1]]).to(device)
            output, hidden, cell = model.decoder(inputs_tensor, hidden, cell)
            predicted_token = output.argmax(-1).item()
            inputs.append(predicted_token)
            if predicted_token == de_vocab[eos_token]:
                break
        tokens = de_vocab.lookup_tokens(inputs)
    return tokens

In [22]:
import evaluate
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load BLEU metric
bleu = evaluate.load("bleu")

# Function to translate sentences from English to German
translations = [
    translate_sentence(
        example["en"],
        model,
        en_nlp,
        de_nlp,
        en_vocab,
        de_vocab,
        False,
        sos_token,
        eos_token,
        device,
    )
    for example in tqdm(test_data)
]

100%|██████████| 1904/1904 [00:47<00:00, 39.84it/s]


In [23]:
# Format the predictions for BLEU evaluation
predictions = ["".join(translation[1:-1]) for translation in translations]
references = [[example["de"]] for example in test_data]

In [24]:
predictions[0], references[0]

('de              e eeeeee',
 ['diese mission erscheint mir als besonders zweckmäßig da die europäische union der wichtigste partner sowohl in bezug auf die entwicklungshilfe als auch auf die wirtschaftliche zusammenarbeit und gleichzeitig der hauptinvestor in peru ist'])

In [25]:
# Calculate BLEU score
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predictions, references=[[ref] for ref in references])

print(f"BLEU score: {results['bleu']:.4f}")

BLEU score: 0.0000


In [26]:
from rouge_score import rouge_scorer

# Adjust the function to ensure `references` is a list of single strings
def compute_rouge(predictions, references):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    
    # Initialize score counters
    scores = {"rouge1": 0, "rouge2": 0, "rougeL": 0}
    count = len(predictions)
    
    for pred, ref in zip(predictions, references):
        # Score method expects both pred and ref to be single strings
        score = scorer.score(ref, pred)
        for key in scores.keys():
            scores[key] += score[key].fmeasure
    
    # Average scores
    for key in scores.keys():
        scores[key] /= count
    
    return scores

# Prepare predictions and references
# Ensure references are in string format
predictions = [" ".join(translation[1:-1]) for translation in translations]
references = [example["de"] for example in test_data]  # Ensure this is a list of strings

# Compute ROUGE scores
rouge_scores = compute_rouge(predictions, references)
print(f"ROUGE scores: {rouge_scores}")

ROUGE scores: {'rouge1': 0.0016402125124592012, 'rouge2': 0.0, 'rougeL': 0.0016173772914142618}


## Load Word2vec embedding Matrices. <br>Train with Word2vec embeddings. <br>Evaluate with Word2vec embeddings.

In [106]:
os.makedirs("embeddings", exist_ok=True)
import gensim

def load_word2vec_embeddings(word2vec_file, vocab, embedding_dim, binary=True):
    # Initialize the embedding matrix with small random values using PyTorch
    embedding_matrix = torch.randn((len(vocab), embedding_dim)) * 0.01
    
    # Get the string-to-index mapping from the vocabulary
    stoi = vocab.get_stoi()
    
    if binary:
        # Load the Word2Vec model from a binary file
        word2vec = gensim.models.KeyedVectors.load_word2vec_format(word2vec_file, binary=True)
        
        # Iterate over each word in the Word2Vec model
        for word in word2vec.key_to_index:
            if word in vocab:  # Check if the word is in the vocabulary
                idx = stoi[word]  # Get the index of the word in the vocabulary
                embedding_matrix[idx] = torch.tensor(word2vec[word])  # Place the embedding at the correct index
    else:
        # Read the embeddings from the text file
        with open(word2vec_file, 'r', encoding='utf-8') as f:
            for line in f:
                parts = line.strip().split()
                # Remove the b' and ' from the word part
                word = parts[0][2:-1]
                vector = torch.tensor([float(x) for x in parts[1:]], dtype=torch.float)
                if word in vocab:  # Check if the word is in the vocabulary
                    idx = stoi[word]  # Get the index of the word in the vocabulary
                    embedding_matrix[idx] = vector  # Place the embedding at the correct index


    # Initialize special token embeddings
    # 0: '<unk>', 1: '<pad>', 2: '<sos>', 3: '<eos>'
    embedding_matrix[0] = torch.randn(embedding_dim) * 0.1  # Small random values for unknown
    embedding_matrix[1] = torch.zeros(embedding_dim)  # Zero initialization for padding token
    embedding_matrix[2] = torch.randn(embedding_dim) * 0.1  # Small random values (sos)
    embedding_matrix[3] = torch.randn(embedding_dim) * 0.1  # Small random values (eos)
    
    return torch.FloatTensor(embedding_matrix)

# Example usage
embedding_dim = 300
en_word2vec_file = 'GoogleNews-vectors-negative300.bin'
de_word2vec_file = 'word2vec.de.txt'

# Assuming en_vocab and de_vocab are your vocabularies of type torchtext.vocab.Vocab
en_wembedding_matrix = load_word2vec_embeddings(en_word2vec_file, en_vocab, embedding_dim, binary=True)
de_wembedding_matrix = load_word2vec_embeddings(de_word2vec_file, de_vocab, embedding_dim, binary=False)

# Save the embedding matrices
# torch.save(en_wembedding_matrix, 'embeddings/en_wembedding_matrix.pt')
# torch.save(de_wembedding_matrix, 'embeddings/de_wembedding_matrix.pt')

# Load the embedding matrices
# en_wembedding_matrix = torch.load("embeddings/en_wembedding_matrix.pt")
# de_wembedding_matrix = torch.load("embeddings/de_wembedding_matrix.pt")

### Train with word2vec embeddings

In [107]:
os.makedirs("models", exist_ok=True)
import gc
gc.collect()
torch.cuda.empty_cache()

model = Seq2Seq(encoder, decoder, device).to(device)

# Load our preptrained word2vec embeddings
encoder.embedding.weight.data.copy_(en_wembedding_matrix)
decoder.embedding.weight.data.copy_(de_wembedding_matrix)

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

n_epochs = 10
clip = 1.0
teacher_forcing_ratio = 0.5
best_valid_loss = float("inf")
for epoch in range(n_epochs):
    train_loss = train_fn(
        model,
        train_data_loader,
        optimizer,
        criterion,
        clip,
        teacher_forcing_ratio,
        device,
    )
    valid_loss = evaluate_fn(
        model,
        valid_data_loader,
        criterion,
        device,
    )
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "models/character_model_word2vec_en_de.pt")
    print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")
    print(f"Batch {epoch+1} done \n")

100%|██████████| 2/2 [00:00<00:00,  3.21it/s]


	Train Loss:   2.926 | Train PPL:  18.653
	Valid Loss:   2.923 | Valid PPL:  18.588
Batch 1 done 



 27%|██▋       | 6/22 [00:04<00:13,  1.22it/s]


KeyboardInterrupt: 

### Evaluate English to German with BLEU and ROUGH scores with Word2Vec embeddings.

In [ ]:
model.load_state_dict(torch.load("models/character_model_word2vec_en_de.pt"))
test_loss = evaluate_fn(model, test_data_loader, criterion, device)

print(f"| Test Loss: {test_loss:.3f} | Test PPL: {np.exp(test_loss):7.3f} |")

In [ ]:
import evaluate
from tqdm import tqdm

# Define special tokens
sos_token = "<sos>"
eos_token = "<eos>"

# Translate the test dataset
translations = [
    translate_sentence(
        example["en"],
        model,
        en_vocab,
        de_vocab,
        sos_token,
        eos_token,
        device,
    )
    for example in tqdm(test_data)
]

In [ ]:
# Format the predictions and references for BLEU evaluation
predictions = ["".join(translation) for translation in translations]
references = ["".join(example["de_tokens"][1:-1]) for example in test_data]

In [ ]:
# Calculate BLEU score
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predictions, references=[[ref] for ref in references])

print(f"BLEU score: {results['bleu']:.4f}")

In [ ]:
from rouge_score import rouge_scorer

# Adjust the function to ensure `references` is a list of single strings
def compute_rouge(predictions, references):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    
    # Initialize score counters
    scores = {"rouge1": 0, "rouge2": 0, "rougeL": 0}
    count = len(predictions)
    
    for pred, ref in zip(predictions, references):
        # Score method expects both pred and ref to be single strings
        score = scorer.score(ref, pred)
        for key in scores.keys():
            scores[key] += score[key].fmeasure
    
    # Average scores
    for key in scores.keys():
        scores[key] /= count
    
    return scores

# Prepare predictions and references
# Ensure references are in string format
predictions = [" ".join(translation[1:-1]) for translation in translations]
references = [example["de"] for example in test_data]  # Ensure this is a list of strings

# Compute ROUGE scores
rouge_scores = compute_rouge(predictions, references)
print(f"ROUGE scores: {rouge_scores}")